
2- Se tiene un registro de transacciones bancarias, de la forma (nro de transacción, tipo de transacción, cuenta origen, cuenta destino, fecha, hora, monto). Se pide resolver en Pandas:

* Validar que todas las transacciones cuenten con un tipo de transacción.
* Validar que para las transacciones del tipo transferencia, exista siempre tanto cuenta origen como cuenta destino.
* Verificar que todas las transacciones del tipo transferencia, depósito y extracción cuenten con montos distintos de cero.
* Indicar cuáles fueron las 10 transacciones de mayor monto.
* Indicar cuál es el tipo de transacción que registra mayor monto promedio.
* Indicar cuáles son las 5 cuentas con mayor cantidad de transacciones.
* Indicar cuáles son las 5 cuentas con mayor monto involucrado.
* Para el tipo de transacción con mayor cantidad de monto promedio, indicar cuales son las 5 cuentas con más transacciones.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_transacciones = pd.read_csv("/DataSets/transacciones.csv")
df_transacciones

,numero,tipo,cuenta_origen,cuenta_destino,fecha,monto
0,991,deposito,18975,15998,18/09/2002,801947
1,1033,transferencia,19109,12682,05/06/2014,563267
2,1001,deposito,12754,12288,25/01/2018,379980
3,1023,NaN,18075,13369,24/10/2008,832748
4,1002,transferencia,17053,18601,24/07/2014,781484
...,...,...,...,...,...,...
95,1024,deposito,17343,12064,27/08/2017,15959
96,1006,extracion,19012,15519,13/02/2016,592950
97,1027,extracion,12133,14223,28/01/2011,68706
98,1010,deposito,11427,12147,04/07/2020,135403


* Validar que todas las transacciones cuenten con un tipo de transacción.

Eliminaremos aquellas filas que no tengan tipo de transaccion. Vemos que realizando la operacion **hasnans** tenemos algunas filas que no tienen tipo de transaccion.

In [ ]:
df_transacciones["tipo"].hasnans

True

In [ ]:
df_transacciones["tipo"].isnull().value_counts()

False    77
True     23
Name: tipo, dtype: int64

In [ ]:
#Con el comando inplace=True modificamos el df original. Entonces nos queda un df sin Nans
df_transacciones.dropna(inplace=True)
df_transacciones.reset_index()

,index,numero,tipo,cuenta_origen,cuenta_destino,fecha,monto
0,0,991,deposito,18975,15998,18/09/2002,801947
1,1,1033,transferencia,19109,12682,05/06/2014,563267
2,2,1001,deposito,12754,12288,25/01/2018,379980
3,4,1002,transferencia,17053,18601,24/07/2014,781484
4,6,1057,transferencia,18829,10213,03/04/2010,246701
...,...,...,...,...,...,...,...
72,95,1024,deposito,17343,12064,27/08/2017,15959
73,96,1006,extracion,19012,15519,13/02/2016,592950
74,97,1027,extracion,12133,14223,28/01/2011,68706
75,98,1010,deposito,11427,12147,04/07/2020,135403


* Validar que para las transacciones del tipo transferencia, exista siempre tanto cuenta origen como cuenta destino.

Nos quedamos solo con las filas que tienen como tipo *transferencia*

In [ ]:
transferencias = df_transacciones.loc[df_transacciones["tipo"] == "transferencia"]
transferencias.head()

,numero,tipo,cuenta_origen,cuenta_destino,fecha,monto
1,1033,transferencia,19109,12682,05/06/2014,563267
4,1002,transferencia,17053,18601,24/07/2014,781484
6,1057,transferencia,18829,10213,03/04/2010,246701
7,1018,transferencia,18750,19918,02/04/2005,141567
14,986,transferencia,15282,12876,29/08/2004,197332


Vemos que aplicando nuevamente la funcion **isnull** tenemos todas las transferencia con cuentas origen y destino

In [ ]:
transferencias[["cuenta_origen","cuenta_destino"]].isnull().any()

cuenta_origen     False
cuenta_destino    False
dtype: bool

* Verificar que todas las transacciones del tipo transferencia, depósito y extracción cuenten con montos distintos de cero.

Comprobamos que ninguna transaccion tenga un monto invalido.

In [ ]:
montos_invalidos = df_transacciones["monto"] < 0
montos_invalidos.value_counts()


False    77
Name: monto, dtype: int64

* Indicar cuáles fueron las 10 transacciones de mayor monto.

In [ ]:
#Con el parametro keep='all' nos quedamos con las filas repetidas ya que para este caso nos importan.
df_transacciones.nlargest(10,"monto",keep='all').reset_index()

,index,numero,tipo,cuenta_origen,cuenta_destino,fecha,monto
0,36,1005,extracion,13849,16480,01/11/2010,970073
1,45,1001,deposito,15016,17727,18/11/2018,937448
2,61,926,transferencia,11337,14355,19/07/2010,930339
3,28,988,deposito,17997,11899,23/11/2008,919017
4,81,990,transferencia,17401,15212,22/11/2011,903401
5,44,1032,deposito,15934,18294,23/03/2017,898985
6,15,1010,extracion,16450,11859,25/12/2007,893868
7,33,994,transferencia,19101,10711,05/03/2016,886957
8,70,982,deposito,14511,18190,02/07/2009,884881
9,21,992,transferencia,14007,13398,26/01/2008,875873


* Indicar cuál es el tipo de transacción que registra mayor monto promedio.

In [ ]:
col_rename = {'monto' : 'monto_promedio'} 
transacciones_agrupadas = df_transacciones.groupby('tipo').agg({'monto' : 'mean'}).rename(columns=col_rename)
transacciones_agrupadas.nlargest(1,'monto_promedio')

,monto_promedio
tipo,
deposito,575641.607143


* Indicar cuáles son las 5 cuentas con mayor cantidad de transacciones.

En este caso nos interesan las cuentas origen que tengan mayor cantidad de transacciones. Entonces buscamos la cuenta que tenga mayor concurrencia en el df (para nuestro caso vemos que son todas cuentas distintas por lo que nos daria las primeras encontradas)

In [ ]:
df_transacciones['cuenta_origen'].value_counts().nlargest(5)

17492    1
14003    1
16732    1
14109    1
12133    1
Name: cuenta_origen, dtype: int64

* Indicar cuáles son las 5 cuentas con mayor monto involucrado.


In [ ]:
mayores_monto = df_transacciones.nlargest(5,'monto').reset_index()
mayores_monto[['cuenta_origen','cuenta_destino','monto']]

,cuenta_origen,cuenta_destino,monto
0,13849,16480,970073
1,15016,17727,937448
2,11337,14355,930339
3,17997,11899,919017
4,17401,15212,903401


* Para el tipo de transacción con mayor cantidad de monto promedio, indicar cuales son las 5 cuentas con más transacciones.